# CVRP Model with Cuckoo Algorithm

Asumsi kasus :
Di sebuah kota X, suatu perusahaan memiliki 1 depot pengiriman barang. Perusahaan ini akan mengirimkan produknya ke Y jumlah pelanggan. Setiap pelanggan memiliki permintaan produk yang berbeda-beda. Perusahaan akan mengirimkan produknya menggunakan Z jumlah kendaraan yang masing-masing kendaraan memiliki kapasitas yang sama sebesar A.

Model ini akan mengeluarkan output total jarak minimum dan rute yang ditempuh oleh setiap kendaraan.Pada setiap rute yang ditempuh akan dicantumkan kapasitas kendaraan yang telah terisi pada rute tersebut.

In [1]:
import numpy as np
import time
start_time = time.time()

Fungsi create_data_model membuat data yang dibutuhkan oleh model

Distance_Matrix : Merupakan data jarak tempuh antara lokasi ke-i dan lokasi ke-j dengan i merupakan indeks kolom dan j merupakan indeks baris. i,j = 0 menggambarkan lokasi depot

Demands : Merupakan data permintaan pelanggan ke-i dengan i merupakan indeks kolom. i = 0 merupakaan depot sehingga permintaan selalu 0

Vehicle_capacity : Merupakan kapasitas maksimum dari setiap kendaraan

Num_vehicles : Merupakan banyaknya kendaraan yang digunakan perusahaan untuk mengirim barang

Depot = Banyaknya depot

In [2]:
# Data model for VRP
def create_data_model():
    data = {}
    data["distance_matrix"] = [
        [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
    ]
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data["vehicle_capacities"] = [15, 15, 15, 15]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data

Fungsi generate_initial_population membuat random route. 
Output dari fungsi ini adalah matriks yang setiap barisnya merupakan solusi yang berisi urutan pelanggan yang akan dikunjungi.
Berikut ini merupakan contoh output dari fungsi ini 
([1, 11, 14, 8,3, 7, 4,15, 12, 2,16, 9, 6])

In [3]:
# Generate initial population
def generate_initial_population(pop_size, num_customers):
    population = []
    for _ in range(pop_size):
        particle = np.random.permutation(num_customers) + 1  # Generate a random route excluding depot
        population.append(particle)
    return np.array(population)

Fungsi form_routes membentuk rute setiap kendaraan dan kapasitas dari setiap kendaraan.
Berikut ini merupakan contoh output dari fungsi ini 
([0, 1, 11, 14, 8, 0], 14), ([0, 3, 7, 4, 0], 14), ([0, 15, 12, 2, 0], 11), ([0, 16, 9, 6, 0], 13)
Dengan array [0, 1, 11, 14, 8, 0] menggambarkan urutan pelanggan yang dikunjungi oleh kendaraan 1 dan 14 merupakan kapasitas  kendaraan 1 yang terisi melalui rute ini

In [4]:
# Route formation for VRP with capacity constraints
def form_routes(data, individual):
    routes = []
    vehicle_capacity = data["vehicle_capacities"]
    demand_data = data["demands"]
    depot = data["depot"]

    assigned_customers = set()
    idx = 0

    for vehicle_id in range(data["num_vehicles"]):
        route = [depot]
        load = 0
        while idx < len(individual):
            customer = individual[idx]
            if customer in assigned_customers:
                idx += 1
                continue

            demand = demand_data[customer]
            if load + demand <= vehicle_capacity[vehicle_id]:
                load += demand
                route.append(customer)
                assigned_customers.add(customer)
                idx += 1
            else:
                break

        route.append(depot)
        routes.append((route, load))

    unassigned_customers = [cust for cust in individual if cust not in assigned_customers]
    for customer in unassigned_customers:
        route = [depot, customer, depot]
        load = demand_data[customer]
        routes.append((route, load))

    return routes

Fungsi calculate_route_distance digunakan untuk menghitung jarak dari setiap rute yang ditempuh menggunakan distance_matrix

In [5]:
# Calculate distance for each route
def calculate_route_distance(route, distance_matrix):
    distance = 0
    for i in range(len(route) - 1):
        distance += distance_matrix[route[i]][route[i + 1]]
    return distance

Fungsi calculate_fitness digunakan untuk menghitung total jarak yang ditempuh oleh semua kendaraan

In [6]:
# Calculate fitness (total distance)
def calculate_fitness(data, individual):
    routes = form_routes(data, individual)
    total_distance = sum(calculate_route_distance(route[0], data["distance_matrix"]) for route in routes)
    return total_distance

Fungsi update_best_nest menentukan solusi terbaik berdasarkan jarak minimum yang ditempuh

In [7]:
# Function: Update best nest
def update_best_nest(position, fitness_values):
    best_idx = np.argmin(fitness_values)
    return position[best_idx], fitness_values[best_idx]

Fungsi levy_flight 

In [8]:
# Function: Levy Flight
def levy_flight(mean):
    u1 = np.random.uniform(-0.5 * np.pi, 0.5 * np.pi)
    u2 = np.random.uniform(-0.5 * np.pi, 0.5 * np.pi)
    v = np.random.uniform(0.0, 1.0)
    x1 = np.sin((mean - 1.0) * u1) / np.power(np.cos(u1), 1.0 / (mean - 1.0))
    x2 = np.power(np.cos((2.0 - mean) * u2) / (-np.log(v)), (2.0 - mean) / (mean - 1.0))
    return x1 * x2

Fungsi cuckoo_search digunakan untuk membetuk solusi terbaik 
Langakh-langkah cuckoo_search :
1. Mengenerate sejumlah urutan pelanggan secara random dalam bentuk matrix menggunakan fungsi generate_initial_population yang akan dianggap sebagai solusi sementara
2. Menghitung jarak dari setiap solusi yang diperoleh pada nomor 1
3. Menentukan solusi terbaik dari beberapa solusi yang dimiliki berdasarkan jarak tempuh terpendek
4. Mengganti urutan pelanggan dari solusi secara random untuk mencari solusi yang lebih baik
6. Mencari solusi terbaik yang baru setelah dilakukan perubahan 

In [9]:
# Function: Cuckoo Search for VRP
def cuckoo_search(data, birds=20, discovery_rate=0.25, alpha_value=0.01, lambda_value=1.5, iterations=2000):
    num_customers = len(data["demands"]) - 1
    position = generate_initial_population(birds, num_customers)
    fitness_values = np.array([calculate_fitness(data, individual) for individual in position])

    best_position, best_fitness = update_best_nest(position, fitness_values)

    for iteration in range(iterations):
        # Replace random birds (solutions) with new solutions
        for i in range(birds):
            new_solution = np.copy(position[i])
            idx1, idx2 = np.random.choice(len(new_solution), 2, replace=False)
            new_solution[idx1], new_solution[idx2] = new_solution[idx2], new_solution[idx1]
            new_fitness = calculate_fitness(data, new_solution)
            if new_fitness < fitness_values[i]:
                position[i] = new_solution
                fitness_values[i] = new_fitness

        # Update positions by abandoning some nests based on discovery rate
        abandoned_nests = int(np.ceil(discovery_rate * birds))
        for _ in range(abandoned_nests):
            bird = np.random.randint(0, birds)
            new_solution = np.random.permutation(num_customers) + 1
            new_fitness = calculate_fitness(data, new_solution)
            if new_fitness < fitness_values[bird]:
                position[bird] = new_solution
                fitness_values[bird] = new_fitness

        # Update the best solution found so far
        current_best_position, current_best_fitness = update_best_nest(position, fitness_values)
        if current_best_fitness < best_fitness:
            best_position, best_fitness = current_best_position, current_best_fitness
        print(f"Iteration {iteration + 1}: Best Distance = {best_fitness}")
    best_routes = form_routes(data, best_position)
    return best_routes, best_fitness

Fungsi print_detailed_solution mengeluarkan output solusi terbaik yang diperoleh 
Berikut contoh output dari fungsi ini
Route for vehicle 1:
 0 Load(0) -> 13 Load(4) -> 15 Load(12) -> 11 Load(13) -> 12 Load(15) -> 0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) -> 7 Load(8) -> 1 Load(9) -> 4 Load(13) -> 3 Load(15) -> 0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) -> 9 Load(1) -> 14 Load(5) -> 16 Load(13) -> 10 Load(15) -> 0 Load(15)
Distance of the route: 1552m
Load of the route: 15

In [10]:
# Print detailed solution
def print_detailed_solution(data, routes, total_distance):
    total_load = 0
    print(f"Objective: {total_distance}")
    for vehicle_id, (route, load) in enumerate(routes):
        route_distance = calculate_route_distance(route, data["distance_matrix"])
        route_load = 0
        output = f"Route for vehicle {vehicle_id + 1}:\n"

        for i in range(len(route) - 1):
            node = route[i]
            demand = data["demands"][node]
            route_load += demand
            output += f" {node} Load({route_load}) ->"
        output += f" {route[-1]} Load({route_load})\n"
        output += f"Distance of the route: {route_distance}m\n"
        output += f"Load of the route: {route_load}\n"

        print(output)
        total_load += route_load

    print(f"Total Distance of all routes: {total_distance}m")
    print(f"Total Load of all routes: {total_load}")


## Main Program

In [11]:
# Main program
def main():
    data = create_data_model()
    best_routes, best_distance = cuckoo_search(data)
    print_detailed_solution(data, best_routes, best_distance)


if __name__ == "__main__":
    main()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime: {elapsed_time} seconds")

Iteration 1: Best Distance = 10704
Iteration 2: Best Distance = 10452
Iteration 3: Best Distance = 9312
Iteration 4: Best Distance = 9312
Iteration 5: Best Distance = 9312
Iteration 6: Best Distance = 9312
Iteration 7: Best Distance = 9312
Iteration 8: Best Distance = 9312
Iteration 9: Best Distance = 9084
Iteration 10: Best Distance = 9084
Iteration 11: Best Distance = 9084
Iteration 12: Best Distance = 9084
Iteration 13: Best Distance = 9084
Iteration 14: Best Distance = 9084
Iteration 15: Best Distance = 9084
Iteration 16: Best Distance = 9084
Iteration 17: Best Distance = 9084
Iteration 18: Best Distance = 9084
Iteration 19: Best Distance = 8904
Iteration 20: Best Distance = 8560
Iteration 21: Best Distance = 8560
Iteration 22: Best Distance = 8560
Iteration 23: Best Distance = 8560
Iteration 24: Best Distance = 8560
Iteration 25: Best Distance = 8560
Iteration 26: Best Distance = 8536
Iteration 27: Best Distance = 8536
Iteration 28: Best Distance = 8536
Iteration 29: Best Distance

Iteration 424: Best Distance = 6208
Iteration 425: Best Distance = 6208
Iteration 426: Best Distance = 6208
Iteration 427: Best Distance = 6208
Iteration 428: Best Distance = 6208
Iteration 429: Best Distance = 6208
Iteration 430: Best Distance = 6208
Iteration 431: Best Distance = 6208
Iteration 432: Best Distance = 6208
Iteration 433: Best Distance = 6208
Iteration 434: Best Distance = 6208
Iteration 435: Best Distance = 6208
Iteration 436: Best Distance = 6208
Iteration 437: Best Distance = 6208
Iteration 438: Best Distance = 6208
Iteration 439: Best Distance = 6208
Iteration 440: Best Distance = 6208
Iteration 441: Best Distance = 6208
Iteration 442: Best Distance = 6208
Iteration 443: Best Distance = 6208
Iteration 444: Best Distance = 6208
Iteration 445: Best Distance = 6208
Iteration 446: Best Distance = 6208
Iteration 447: Best Distance = 6208
Iteration 448: Best Distance = 6208
Iteration 449: Best Distance = 6208
Iteration 450: Best Distance = 6208
Iteration 451: Best Distance

Iteration 656: Best Distance = 6208
Iteration 657: Best Distance = 6208
Iteration 658: Best Distance = 6208
Iteration 659: Best Distance = 6208
Iteration 660: Best Distance = 6208
Iteration 661: Best Distance = 6208
Iteration 662: Best Distance = 6208
Iteration 663: Best Distance = 6208
Iteration 664: Best Distance = 6208
Iteration 665: Best Distance = 6208
Iteration 666: Best Distance = 6208
Iteration 667: Best Distance = 6208
Iteration 668: Best Distance = 6208
Iteration 669: Best Distance = 6208
Iteration 670: Best Distance = 6208
Iteration 671: Best Distance = 6208
Iteration 672: Best Distance = 6208
Iteration 673: Best Distance = 6208
Iteration 674: Best Distance = 6208
Iteration 675: Best Distance = 6208
Iteration 676: Best Distance = 6208
Iteration 677: Best Distance = 6208
Iteration 678: Best Distance = 6208
Iteration 679: Best Distance = 6208
Iteration 680: Best Distance = 6208
Iteration 681: Best Distance = 6208
Iteration 682: Best Distance = 6208
Iteration 683: Best Distance

Iteration 901: Best Distance = 6208
Iteration 902: Best Distance = 6208
Iteration 903: Best Distance = 6208
Iteration 904: Best Distance = 6208
Iteration 905: Best Distance = 6208
Iteration 906: Best Distance = 6208
Iteration 907: Best Distance = 6208
Iteration 908: Best Distance = 6208
Iteration 909: Best Distance = 6208
Iteration 910: Best Distance = 6208
Iteration 911: Best Distance = 6208
Iteration 912: Best Distance = 6208
Iteration 913: Best Distance = 6208
Iteration 914: Best Distance = 6208
Iteration 915: Best Distance = 6208
Iteration 916: Best Distance = 6208
Iteration 917: Best Distance = 6208
Iteration 918: Best Distance = 6208
Iteration 919: Best Distance = 6208
Iteration 920: Best Distance = 6208
Iteration 921: Best Distance = 6208
Iteration 922: Best Distance = 6208
Iteration 923: Best Distance = 6208
Iteration 924: Best Distance = 6208
Iteration 925: Best Distance = 6208
Iteration 926: Best Distance = 6208
Iteration 927: Best Distance = 6208
Iteration 928: Best Distance

Iteration 1135: Best Distance = 6208
Iteration 1136: Best Distance = 6208
Iteration 1137: Best Distance = 6208
Iteration 1138: Best Distance = 6208
Iteration 1139: Best Distance = 6208
Iteration 1140: Best Distance = 6208
Iteration 1141: Best Distance = 6208
Iteration 1142: Best Distance = 6208
Iteration 1143: Best Distance = 6208
Iteration 1144: Best Distance = 6208
Iteration 1145: Best Distance = 6208
Iteration 1146: Best Distance = 6208
Iteration 1147: Best Distance = 6208
Iteration 1148: Best Distance = 6208
Iteration 1149: Best Distance = 6208
Iteration 1150: Best Distance = 6208
Iteration 1151: Best Distance = 6208
Iteration 1152: Best Distance = 6208
Iteration 1153: Best Distance = 6208
Iteration 1154: Best Distance = 6208
Iteration 1155: Best Distance = 6208
Iteration 1156: Best Distance = 6208
Iteration 1157: Best Distance = 6208
Iteration 1158: Best Distance = 6208
Iteration 1159: Best Distance = 6208
Iteration 1160: Best Distance = 6208
Iteration 1161: Best Distance = 6208
I

Iteration 1403: Best Distance = 6208
Iteration 1404: Best Distance = 6208
Iteration 1405: Best Distance = 6208
Iteration 1406: Best Distance = 6208
Iteration 1407: Best Distance = 6208
Iteration 1408: Best Distance = 6208
Iteration 1409: Best Distance = 6208
Iteration 1410: Best Distance = 6208
Iteration 1411: Best Distance = 6208
Iteration 1412: Best Distance = 6208
Iteration 1413: Best Distance = 6208
Iteration 1414: Best Distance = 6208
Iteration 1415: Best Distance = 6208
Iteration 1416: Best Distance = 6208
Iteration 1417: Best Distance = 6208
Iteration 1418: Best Distance = 6208
Iteration 1419: Best Distance = 6208
Iteration 1420: Best Distance = 6208
Iteration 1421: Best Distance = 6208
Iteration 1422: Best Distance = 6208
Iteration 1423: Best Distance = 6208
Iteration 1424: Best Distance = 6208
Iteration 1425: Best Distance = 6208
Iteration 1426: Best Distance = 6208
Iteration 1427: Best Distance = 6208
Iteration 1428: Best Distance = 6208
Iteration 1429: Best Distance = 6208
I

Iteration 1673: Best Distance = 6208
Iteration 1674: Best Distance = 6208
Iteration 1675: Best Distance = 6208
Iteration 1676: Best Distance = 6208
Iteration 1677: Best Distance = 6208
Iteration 1678: Best Distance = 6208
Iteration 1679: Best Distance = 6208
Iteration 1680: Best Distance = 6208
Iteration 1681: Best Distance = 6208
Iteration 1682: Best Distance = 6208
Iteration 1683: Best Distance = 6208
Iteration 1684: Best Distance = 6208
Iteration 1685: Best Distance = 6208
Iteration 1686: Best Distance = 6208
Iteration 1687: Best Distance = 6208
Iteration 1688: Best Distance = 6208
Iteration 1689: Best Distance = 6208
Iteration 1690: Best Distance = 6208
Iteration 1691: Best Distance = 6208
Iteration 1692: Best Distance = 6208
Iteration 1693: Best Distance = 6208
Iteration 1694: Best Distance = 6208
Iteration 1695: Best Distance = 6208
Iteration 1696: Best Distance = 6208
Iteration 1697: Best Distance = 6208
Iteration 1698: Best Distance = 6208
Iteration 1699: Best Distance = 6208
I

Iteration 1961: Best Distance = 6208
Iteration 1962: Best Distance = 6208
Iteration 1963: Best Distance = 6208
Iteration 1964: Best Distance = 6208
Iteration 1965: Best Distance = 6208
Iteration 1966: Best Distance = 6208
Iteration 1967: Best Distance = 6208
Iteration 1968: Best Distance = 6208
Iteration 1969: Best Distance = 6208
Iteration 1970: Best Distance = 6208
Iteration 1971: Best Distance = 6208
Iteration 1972: Best Distance = 6208
Iteration 1973: Best Distance = 6208
Iteration 1974: Best Distance = 6208
Iteration 1975: Best Distance = 6208
Iteration 1976: Best Distance = 6208
Iteration 1977: Best Distance = 6208
Iteration 1978: Best Distance = 6208
Iteration 1979: Best Distance = 6208
Iteration 1980: Best Distance = 6208
Iteration 1981: Best Distance = 6208
Iteration 1982: Best Distance = 6208
Iteration 1983: Best Distance = 6208
Iteration 1984: Best Distance = 6208
Iteration 1985: Best Distance = 6208
Iteration 1986: Best Distance = 6208
Iteration 1987: Best Distance = 6208
I